In [0]:
%sql
WITH last_month_orders AS (
  SELECT
    dp.product_name,
    total_quantity,
    order_date
  FROM
    sales_prd.rafacferrez_gold.fact_daily_order fdo
      INNER JOIN sales_prd.rafacferrez_gold.dim_product dp
        ON fdo.product_id = dp.product_id
  WHERE date_trunc('MONTH', order_date) = date_trunc('MONTH', current_date())
)
SELECT
  product_name,
  sum(total_quantity) as product_total_qty
FROM
  last_month_orders
  GROUP BY
    product_name
ORDER BY
  product_total_qty DESC
LIMIT 1

In [0]:
%sql
WITH last_month_orders AS (
  SELECT
    product_id,
    total_quantity
  FROM
    sales_prd.rafacferrez_gold.fact_daily_order
  WHERE
    date_trunc('MONTH', order_date) = date_trunc('MONTH', current_date())
),
orders_by_product AS (
  SELECT
    dp.product_name,
    sum(total_quantity) as product_total_qty
  FROM
    last_month_orders fdo
      INNER JOIN sales_prd.rafacferrez_gold.dim_product dp
        ON fdo.product_id = dp.product_id
  GROUP BY
    dp.product_name
)
SELECT
  product_name,
  product_total_qty
FROM
  orders_by_product
QUALIFY
  DENSE_RANK() OVER (ORDER BY product_total_qty DESC) = 1;